### Databricks Stored Procedure Example

In [0]:
%sql
CREATE PROCEDURE dev_catalog.bronze.calc_sum(
  IN a INT, 
  IN b INT, 
  OUT sum INT, 
  INOUT total INT
) 
LANGUAGE SQL 
SQL SECURITY INVOKER
AS
BEGIN
  SET sum = a + b;
  SET total = total + sum;
END;


In [0]:
%sql
DECLARE a INT DEFAULT 0;
DECLARE b INT DEFAULT 0;
CALL dev_catalog.bronze.calc_sum(5, 7, a, b);  -- a becomes 12, b becomes 12
SELECT a, b;

a,b
12,12


### Databricks Sample Stored Procedures - Use Cases Implementations

#### Initialize Data

In [0]:
%sql
-- Create Table sales
CREATE OR REPLACE TABLE dev_catalog.bronze.sales_raw  (
  sale_id INT,
  sale_date DATE,
  product STRING,
  region STRING,
  amount DOUBLE
);

-- Insert sample data
INSERT INTO dev_catalog.bronze.sales_raw VALUES
  (1, '2025-01-15', 'Laptop', 'North', 1200.55),
  (2, '2025-01-17', 'Laptop', 'South', 1150.00),
  (3, '2025-02-10', 'Mouse', 'North', 20.75),
  (4, '2025-02-11', 'Mouse', 'West', 21.25),
  (5, '2025-03-05', 'Keyboard', 'North', 35.00),

  (6, '2025-03-12', 'Monitor', 'East', 250.00),
  (7, '2025-03-20', 'Laptop', 'West', 1185.75),
  (8, '2025-03-25', 'Keyboard', 'South', 34.50),

  (9, '2025-04-02', 'Mouse', 'East', 19.95),
  (10, '2025-04-18', 'Monitor', 'North', 245.00),

  (11, '2025-05-05', 'Laptop', 'East', 1225.00),
  (12, '2025-05-10', 'Keyboard', 'West', 36.25),
  (13, '2025-05-14', 'Mouse', 'South', 22.10),
  (14, '2025-05-25', 'Monitor', 'South', 255.75),

  (15, '2025-06-02', 'Laptop', 'North', 1199.99),
  (16, '2025-06-08', 'Keyboard', 'East', 35.75),
  (17, '2025-06-15', 'Monitor', 'West', 248.20),
  (18, '2025-06-22', 'Mouse', 'North', 21.10),

  (19, '2025-07-03', 'Laptop', 'West', 1170.50),
  (20, '2025-07-09', 'Mouse', 'East', 20.40),
  (21, '2025-07-14', 'Monitor', 'North', 249.99),
  (22, '2025-07-27', 'Keyboard', 'South', 34.80),

  (23, '2025-08-05', 'Laptop', 'South', 1188.25),
  (24, '2025-08-11', 'Mouse', 'West', 19.85),
  (25, '2025-08-17', 'Keyboard', 'North', 36.10),
  (26, '2025-08-28', 'Monitor', 'East', 251.35);



num_affected_rows,num_inserted_rows
26,26


In [0]:
%sql
-- Create cleaned & log tables
CREATE OR REPLACE TABLE dev_catalog.bronze.sales_cleaned (
  sale_id INT,
  sale_date DATE,
  product STRING,
  region STRING,
  amount DOUBLE
);

CREATE OR REPLACE TABLE dev_catalog.bronze.etl_log (
  proc_name STRING,
  run_timestamp TIMESTAMP,
  rows_processed INT
);

#### Simple Stored Procedure — Data Cleaning & Logging
- Clean and load monthly data from sales_raw → sales_cleaned, then log the operation
- Use Case: ETL step that cleans and logs monthly batches.

In [0]:
%sql
CREATE OR REPLACE PROCEDURE dev_catalog.bronze.sp_load_monthly_sales(
  IN p_year INT, 
  IN p_month INT
)
SQL SECURITY INVOKER
LANGUAGE SQL
AS
BEGIN
  -- Load only the required month's data
  INSERT INTO dev_catalog.bronze.sales_cleaned
  SELECT *
  FROM dev_catalog.bronze.sales_raw
  WHERE YEAR(sale_date) = p_year AND MONTH(sale_date) = p_month;

  -- Clean data 
  UPDATE dev_catalog.bronze.sales_cleaned
  SET amount = ROUND(amount, 2);

  -- Log run
  INSERT INTO dev_catalog.bronze.etl_log
  SELECT 
    'sp_load_monthly_sales' AS procedure_name,
    CURRENT_TIMESTAMP() AS run_time,
    COUNT(*) AS record_count
  FROM dev_catalog.bronze.sales_cleaned
  WHERE YEAR(sale_date) = p_year AND MONTH(sale_date) = p_month;
END;


In [0]:
%sql
CALL  dev_catalog.bronze.sp_load_monthly_sales(2025, 8);


In [0]:
%sql
select *
from dev_catalog.bronze.etl_log

proc_name,run_timestamp,rows_processed
sp_load_monthly_sales,2025-10-19T20:08:25.128Z,4
sp_load_monthly_sales,2025-10-19T20:08:40.731Z,4
sp_load_monthly_sales,2025-10-19T20:07:54.044Z,4
sp_load_monthly_sales,2025-10-19T20:08:34.081Z,4
sp_load_monthly_sales,2025-10-19T20:07:47.342Z,4
sp_load_monthly_sales,2025-10-19T20:08:48.103Z,4
sp_load_monthly_sales,2025-10-19T20:08:06.838Z,2
sp_load_monthly_sales,2025-10-19T20:08:00.736Z,5
sp_load_monthly_sales,2025-10-19T20:08:16.495Z,4


#### Stored Procedure — Aggregation and Reporting
- Generate a monthly sales summary by region and product.
- Use Case: Executing analytical summaries inside Databricks SQL dashboards.

In [0]:
%sql
CREATE OR REPLACE PROCEDURE dev_catalog.bronze.sp_monthly_sales_summary(
  IN p_year INT
)
SQL SECURITY INVOKER
LANGUAGE SQL
AS
BEGIN
  DECLARE SQL_STMT STRING;
  
  SET SQL_STMT = '
    CREATE OR REPLACE VIEW dev_catalog.bronze.vw_monthly_summary_' || p_year || ' AS
    SELECT 
      YEAR(sale_date) AS year,
      MONTH(sale_date) AS month,
      region,
      product,
      ROUND(SUM(amount), 2) AS total_sales,
      COUNT(*) AS num_transactions
    FROM dev_catalog.bronze.sales_cleaned
    WHERE YEAR(sale_date) = ' || p_year || '
    GROUP BY year, month, region, product
    ORDER BY month, region';
  
  EXECUTE IMMEDIATE SQL_STMT;
END;

In [0]:
%sql
CALL dev_catalog.bronze.sp_monthly_sales_summary(2025);


In [0]:
%sql
select * from dev_catalog.bronze.vw_monthly_summary_2025

year,month,region,product,total_sales,num_transactions
2025,1,North,Laptop,2401.1,2
2025,1,South,Laptop,2300.0,2
2025,2,North,Mouse,41.5,2
2025,2,West,Mouse,42.5,2
2025,3,East,Monitor,250.0,1
2025,3,North,Keyboard,70.0,2
2025,3,South,Keyboard,34.5,1
2025,3,West,Laptop,1185.75,1
2025,4,East,Mouse,39.9,2
2025,4,North,Monitor,490.0,2


#### Stored Procedure — Dynamic Pivot Query
- Create a dynamic pivot report where months become columns.
- When month columns vary year-to-year — used in dynamic reporting dashboards.

In [0]:
%sql
CREATE OR REPLACE PROCEDURE dev_catalog.bronze.sp_sales_pivot(
  IN p_year INT
)
SQL SECURITY INVOKER
LANGUAGE SQL
AS
BEGIN
  DECLARE month_list STRING;
  DECLARE pivot_query STRING;

  -- Build comma-separated month list dynamically
  SET month_list = (
    SELECT LISTAGG(DISTINCT CONCAT('\'', DATE_FORMAT(sale_date, 'MMMM'), '\''), ', ')
    FROM dev_catalog.bronze.sales_cleaned 
    WHERE YEAR(sale_date) = p_year
  );

  -- Build dynamic pivot SQL
  SET pivot_query = '
    SELECT * FROM (
      SELECT 
        region, 
        product,
        DATE_FORMAT(sale_date, "MMMM") as month,
        amount
      FROM dev_catalog.bronze.sales_cleaned 
      WHERE YEAR(sale_date) = ' || p_year || '
    )
    PIVOT (
      SUM(amount) FOR month IN (' || month_list || ')
    )
    ORDER BY region, product
  ';

  -- Execute it
  EXECUTE IMMEDIATE pivot_query;
END;


In [0]:
%sql
CALL dev_catalog.bronze.sp_sales_pivot(2025);

region,product,June,April,May,January,February,March,July,August
East,Keyboard,35.75,null,null,null,null,null,null,null
East,Laptop,null,null,1225.0,null,null,null,null,null
East,Monitor,null,null,null,null,null,250.0,null,251.35
East,Mouse,null,39.9,null,null,null,null,20.4,null
North,Keyboard,null,null,null,null,null,70.0,null,36.1
North,Laptop,1199.99,null,null,2401.1,null,null,null,null
North,Monitor,null,490.0,null,null,null,null,249.99,null
North,Mouse,21.1,null,null,null,41.5,null,null,null
South,Keyboard,null,null,null,null,null,34.5,34.8,null
South,Laptop,null,null,null,2300.0,null,null,null,1188.25


#### Data Engineering SP — Parameterized ETL Job with Logging
- End-to-end data refresh and audit tracking.
- A production-ready stored procedure that performs all ETL steps and logs outcomes.

In [0]:
%sql
CREATE OR REPLACE PROCEDURE dev_catalog.bronze.sp_etl_refresh(
  IN p_year INT, 
  IN p_month INT)
SQL SECURITY INVOKER
LANGUAGE SQL
AS
BEGIN
  DECLARE rows_loaded INT;

  -- Purge old month data
  DELETE FROM dev_catalog.bronze.sales_cleaned WHERE YEAR(sale_date)=p_year AND MONTH(sale_date)=p_month;

  -- Load data from raw
  INSERT INTO dev_catalog.bronze.sales_cleaned
  SELECT * FROM dev_catalog.bronze.sales_raw
  WHERE YEAR(sale_date)=p_year AND MONTH(sale_date)=p_month;

  -- Capture row count
  SET rows_loaded = (
    SELECT COUNT(*) 
    FROM dev_catalog.bronze.sales_cleaned 
    WHERE YEAR(sale_date)=p_year AND MONTH(sale_date)=p_month
    );

  -- Log operation
  INSERT INTO dev_catalog.bronze.etl_log VALUES ('sp_etl_refresh', CURRENT_TIMESTAMP(), rows_loaded);

  -- Output result
  SELECT CONCAT('ETL completed successfully. Rows loaded: ', rows_loaded) AS message;
END;


In [0]:
%sql
CALL dev_catalog.bronze.sp_etl_refresh(2025, 3);

message
ETL completed successfully. Rows loaded: 5
